In [3]:
import time
import numpy as np
import matplotlib.pyplot as plt
from ripser import ripser
from persim import plot_diagrams
from sklearn.cluster import KMeans
import tadasets
from itertools import combinations
%matplotlib inline

In [17]:
#this is the triangulation of the torus

X = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [0,1], [0,3], [1,3],
    [0,1,3], [1,5], [3,5], [1,3,5], [1,2], [2,5], [1,2,5], [2,6], [5,6], [2,5,6], 
    [0,2], [0,6], [0,2,6], [3,6], [0,3,6], [3,4], [4,5], [3,4,5], [5,7], [4,7], 
    [4,5,7], [6,7], [5,6,7], [6,8], [7,8], [6,7,8], [3,8], [3,6,8], [4,8], 
    [3,4,8], [0,4], [0,7], [0,4,7],
    [1,7], [0,1,7], [1,8], [1,7,8], [2,8], [1,2,8], [2,4], [2,4,8], [0,2,4]] 


X_prime = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [0,1], [0,3], [1,3],
    [0,1,3], [1,5], [3,5], [1,3,5], [1,2], [2,5], [1,2,5], [2,6], [5,6], [2,5,6], 
    [0,2], [0,6], [0,2,6], [3,6], [0,3,6], [3,4], [4,5], [3,4,5], [5,7], [4,7], 
    [4,5,7], [6,7], [5,6,7], [6,8], [7,8], [6,7,8], [3,8], [3,6,8], [4,8], 
    [3,4,8], [0,4], [0,7], [0,4,7],
    [1,7], [0,1,7], [1,8], [1,7,8], [2,8], [1,2,8], [2,4], [2,4,8], [0,2,4],[0,1,2], [2,3],
          [0,2,3], [1,2,3]] 

Y = [[0], [1], [2], [0,1], [0,2], [1,2], [0,1,2]]

print(len(X))

54


In [5]:
# these are helper functions to get the boundary map 
def get_faces(lst):
    return [lst[:i] + lst[i+1:] for i in range(len(lst))]


def get_coeff(simplex, faces):
    if simplex in faces:
        idx = faces.index(simplex)
        return 1 if idx%2==0 else -1
    else:
        return 0
    
    
def tot_bnd(complex): 
    cols = []
    for simplex in complex:
        if len(simplex) == 1:
            bound = [0] * len(complex)
            cols.append(bound)
        if len(simplex) == 2:
            bound = [0] * len(complex)
            bound[complex.index([simplex[0]])] = -1
            bound[complex.index([simplex[1]])] = 1
            cols.append(bound)
        if len(simplex) == 3:
            bound = [0] * len(complex)
            bound[complex.index(simplex[:-1])] = 1
            bound[complex.index(simplex[1:])] = -1
            bound[complex.index([simplex[0]] + [simplex[2]])] = 1
            cols.append(bound)
        if len(simplex) == 4:
            bound = [0] * len(complex)
            bound[complex.index(simplex[1:])] = 1
            bound[complex.index([simplex[0]] + simplex[2:])] = -1
            bound[complex.index(simplex[:2] + [simplex[3]])] = 1  
            bound[complex.index(simplex[:-1])] = -1
            cols.append(bound)
    return np.array(cols).T


def boundary(complex):
    maxdim = len(max(complex, key=len))
    simplices = [sorted([spx for spx in complex if len(spx)==i]) for i in range(1,maxdim+1)]
    bnd = []
    for spx_k, spx_kp1 in zip(simplices, simplices[1:]):
        mtx = []
        for sigma in spx_kp1:
            faces = get_faces(sigma)
            mtx.append([get_coeff(spx, faces) for spx in spx_k])
        bnd.append(np.array(mtx).T)

    return bnd

In [6]:
np.set_printoptions(linewidth=700)

print(tot_bnd(X))
print(len(tot_bnd(X)))

        
for i in range(len(tot_bnd(X))):
    print(tot_bnd(X)[i])


[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ... -1  0  0]
 ...
 [ 0  0  0 ...  0  1 -1]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
54
[ 0  0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  1  0 -1  0 -1  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -1  0 -1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  1  1  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -1  0  0 -1  0  0  0  0  0  0  0  0 -1  0  1  0  0  0  0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0  0

In [24]:
#bnd_2 =bnd_X[1]
#np.set_printoptions(linewidth=110)
#print(bnd_2)
#print(bnd_2.T)

#print(len(bnd_2))



In [8]:
def col_reduce(matrix, c1, c2, row_entry, pivot_rows, pivot_entries, V):
    scale = (matrix[row_entry][c2]//matrix[row_entry][c1])* -(matrix[row_entry][c1]//matrix[row_entry][c1])
    new_row, new_entry = float("inf"), 0
    for i in range(len(matrix)):
        matrix[i][c2] += scale * matrix[i][c1]
        V[i][c2] -= scale * V[i][c1]
        if matrix[i][c2] != 0:
            new_row = i
            new_entry = matrix[i][c2]
    pivot_rows[c2] = new_row
    pivot_entries[c2]  = new_entry



def red_al(matrix): #called Kim Func before
    pivot_rows = []
    pivot_entries =[]
    pivot = False
    for j in range(len(matrix[0])):
        for i in reversed(range(len(matrix))):
            if matrix[i][j] != 0:
                pivot_rows.append(i)
                pivot_entries.append(matrix[i][j])
                pivot = True
                break
        if not pivot:
            pivot_rows.append(float("inf"))
            pivot_entries.append(0)
            pivot = False
    V = np.eye(len(matrix))        
    for i in range(1, len(pivot_rows)):
        while pivot_rows[i] in pivot_rows[0:i] and pivot_rows[i] != float("inf"):
            col = pivot_rows[0:i].index(pivot_rows[i])
            col_reduce(matrix, col, i, pivot_rows[i], pivot_rows, pivot_entries, V)
            
    return matrix, V

In [9]:
np.set_printoptions(linewidth=210)
print(red_al(tot_bnd(X)))
print(red_al(tot_bnd(X)))

mat_R_V = red_al(tot_bnd(X))
        
for i in range(len(mat_R_V[0])):
    print(mat_R_V[0][i])



(array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1., -1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]]))
[ 0  0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0 -1  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  

In [10]:
#Convert Boundary Matrix into Coboundary
coR = np.flip(mat_R_V[0]).T
coV = np.flip(mat_R_V[1]).T
cobnd = np.flip(tot_bnd(X)).T

In [11]:
print (coR)
print (coV)
print (cobnd)

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1.  0. ...  0.  0.  0.]
 [ 0.  0.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 0.  0.  0. ...  0.  1.  0.]
 [ 0.  0.  0. ...  0.  0.  1.]]
[[ 0  0 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 [ 0  0  0 ... -1  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [12]:
# now we want to get the zero columns of coR and the corresponding columns of coV

def get_zero_columns(matrix):
    zero_columns = {}
    num_rows = len(matrix)
    num_columns = len(matrix[0])

    for j in range(num_columns):
        column = [matrix[i][j] for i in range(num_rows)]
        if all(element == 0 for element in column):
            zero_columns[j] = column
    return zero_columns



zero_columns = get_zero_columns(coR)
print(zero_columns)
print(len(zero_columns))

{0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 5: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 7: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [13]:
zero_columns.keys()
zero_cols= list(zero_columns.keys())
print(zero_cols)

[0, 1, 3, 5, 7, 9, 12, 14, 16, 19, 21, 24, 27, 29, 32, 35, 38, 41]


In [14]:
def get_cocycl_col(matrix):
    cocycle_vals = []
    for i in zero_cols:
        cocycle_vals.append(matrix.T[i])
        
    return cocycle_vals   
np.set_printoptions(linewidth=210)
get_cocycl_col(coV)

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]),
 array([-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([-1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [16]:
len(get_cocycl_col(coV))
len(get_cocycl_col(coV)[1])

54

In [23]:
result = ripser(tot_bnd(X), distance_matrix = True, coeff = 2, do_cocycles = True)
diagrams = result['dgms']
cocycles = result['cocycles'] 
